In [23]:
!pip install ijson

In [24]:
import json
import ijson
from google.colab import files  # Only needed if running in Google Colab

In [25]:
def split_json(filename):
    with open(filename, 'r', encoding='utf-8') as f:  # Add encoding for potential issues
        # Use a prefix that targets the specific structure if known
        # For example, if your file is a list of objects:
        objects = ijson.items(f, 'item')
        # If it's a dictionary with a key 'data' containing the list:
        # objects = ijson.items(f, 'data.item')
        batch = []
        file_index = 0
        output_prefix = '{0}_split_file'.format(filename)
        batch_size = 10000
        for obj in objects:
            batch.append(obj)
            # When batch size is met, write to a new file
            if len(batch) >= batch_size:
                with open(f"{output_prefix}_{file_index}.json", 'w', encoding='utf-8') as outfile: # Add encoding here too
                    json.dump(batch, outfile, ensure_ascii=False, indent=2)  # Ensure proper encoding and formatting
                batch = []
                file_index += 1

        # Write remaining data if any
        if batch:
            with open(f"{output_prefix}_{file_index}.json", 'w', encoding='utf-8') as outfile: # Add encoding here too
                json.dump(batch, outfile, ensure_ascii=False, indent=2)  # Ensure proper encoding and formatting
        return file_index + 1

In [26]:
def addControversialUsersComments(file, count, controversial_user_comments):
  for i in range(count):
    with open(f'{file}_split_file_{i}.json', 'r') as json_file:
      batch = json.load(json_file)
      for comment in batch:
        if comment['controversial']==1:
          controversial_user_comments.append(comment)

In [ ]:
UserComments_files = ['2023_8.json', '2023_9.json', '2023_10.json', '2023_11.json']
controversial_user_comments = list()

for file in UserComments_files:

  print(f'Splitting {file}')
  filesCount = split_json(file)

  print(f'Files created: {filesCount}')
  for i in range(filesCount):
    files.download(f'{file}_split_file_{i}.json')

  addControversialUsersComments(file, filesCount, controversial_user_comments)
  print(f'Controversial users comments: {len(controversial_user_comments)}')

In [28]:
with open(f'controversial_users_comments.json', 'w') as json_file:
        json.dump(controversial_user_comments, json_file)